In [5]:
import requests
import csv
import time

In [6]:
def get_binance_data(symbol, interval, start_time, end_time=None):
    url = "https://api.binance.com/api/v3/klines"
    limit = 500  # Limite imposée par l'API Binance
    data = []

    print("\n Collecte des données...")
    while True:
        params = {
            'symbol': symbol,
            'interval': interval,
            'limit': limit,
            'startTime': start_time,
            'endTime': end_time
        }
        response = requests.get(url, params=params)
        new_data = response.json()
        
        if len(new_data) == 0:
            break  # Sort de la boucle s'il n'y a plus de données à récupérer

        data.extend(new_data)  # Ajoute les nouvelles données à la liste totale

        # Récupère le dernier timestamp de l'appel actuel pour la prochaine itération
        start_time = new_data[-1][0] + 1  # Incrémenter pour ne pas répéter le dernier point
        
        # Délai pour éviter de dépasser le taux limite de l'API (limite de 1200 requêtes par minute)
        time.sleep(0.05)
    return data

def get_coef_data(data):
    d_data = []
    previous_row = []
    for i, row in enumerate(data):
        if i > 0:
            d_data.append([row[0], 
                      float(row[1])/float(previous_row[1]),
                      float(row[2])/float(previous_row[2]),
                      float(row[3])/float(previous_row[3]),
                      float(row[4])/float(previous_row[4]),
                      row[5],
                      row[6]
                      ])
        previous_row = row
    return d_data
        

def save_to_csv(data, filename):
    # Définir les noms des colonnes
    headers = ['Open_time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close_time']

    # Ouvrir ou créer le fichier CSV
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        
        # Écrire l'en-tête
        writer.writerow(headers)
        
        # Écrire les lignes de données
        for row in data:
            row = row[:7]
            # Convertir les timestamps en format lisible si besoin
            #row[0] = time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(row[0] / 1000))  # Ouverture
            #row[6] = time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(row[6] / 1000))  # Fermeture
            writer.writerow(row)

In [7]:
pair = 'ETHUSDT'
interval = '15m'
start = 0
end = None


data = get_binance_data(pair, interval, start, end)
d_data = get_coef_data(data)

save_to_csv(data, f"binance_data_of_{pair}_{interval}_{start}_{end}.csv")
save_to_csv(d_data, f"binance_coef_data_of_{pair}_{interval}_{start}_{end}.csv")
print("Données sauvegardée(s) (.csv)")


 Collecte des données...
Données sauvegardée(s) (.csv)
